# Logistic Regression

Example that explains the logistic regression ideas.

In this example we'll be using

* `NumPy`
* `pandas`
* `matplotlib`

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
import io

%matplotlib inline

Let's read the dataset first. It was taken from [Andrew Ng’s Machine Learning course on Coursera](https://github.com/animesh-agarwal/Machine-Learning/blob/master/LogisticRegression/data/marks.txt).

It contains marks of two exams for 100 applicants. The target value take `0` or `1` (was applicant admitted or not)

In [2]:
url = "https://raw.githubusercontent.com/animesh-agarwal/Machine-Learning/master/LogisticRegression/data/marks.txt"
content = requests.get(url).content

data = pd.read_csv(io.StringIO(content.decode('utf-8')), header=None)
data.head()

,0,1,2
0,34.623660,78.024693,0
1,30.286711,43.894998,0
2,35.847409,72.902198,0
3,60.182599,86.308552,1
4,79.032736,75.344376,1


Let's separate features from target labels. And then try to visualize two classes of applicants

In [5]:
X = data.iloc[:, :-1]
y_label = data[:, -1]

admitted = data.loc[y_label == 1]
not_admitted = data.loc[y_label == 0]

plt.scatter(admitted.iloc[:, 0], admitted.iloc[:, 1], s=10, label='Admitted')
plt.scatter(not_admitted.iloc[:, 0], not_admitted.iloc[:, 1], s=10, label='Not Admitted')
plt.legend()

TypeError: '(slice(None, None, None), -1)' is an invalid key